In [1]:
import os
import shutil
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras import layers, models
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,
                             shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Validation data should not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
from keras.applications import VGG16

In [6]:
conv_base = VGG16 (weights = 'imagenet', include_top = False, input_shape = (150 ,150, 3))

58889256/58889256 [==============================] - 4s 0us/step


In [7]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [8]:
base_dir = 'E:\Keras\Dataset\dogs-vs-cats_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [9]:
datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 20

In [10]:
def features_extract (directory, sample_count):
    features = np.zeros(shape = (sample_count, 4, 4, 512))
    labels = np.zeros(shape = (sample_count))
    generator = datagen.flow_from_directory(directory, target_size= (150, 150), batch_size = batch_size,
                                            class_mode = 'binary')

    i = 0
    for input_batch, labels_batch in generator:
        features_batch = conv_base.predict(input_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [11]:
model3 = models.Sequential()
model3.add(conv_base)
model3.add(layers.Flatten())
model3.add(layers.Dense(256, activation = 'relu'))
model3.add(layers.Dense(1, activation = 'sigmoid'))

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,
                             shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest')

# Validation data should not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/train', target_size=(150,150), batch_size=20, class_mode= 'binary')

Found 557 images belonging to 2 classes.


In [14]:
validation_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/test',
                                                        target_size=(150,150),
                                                        batch_size=32,
                                                        class_mode= 'binary')

Found 140 images belonging to 2 classes.


In [15]:
model3.compile(optimizer = RMSprop(learning_rate=0.00001), loss = 'binary_crossentropy', metrics= ['acc'])

In [16]:
history = model3.fit_generator(train_generator, steps_per_epoch = 20, epochs = 20,
                               validation_data = validation_generator, validation_steps = 5)

<ipython-input-16-da2e4232ac55>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model3.fit_generator(train_generator, steps_per_epoch = 20, epochs = 20,


Epoch 1/20
20/20 [==============================] - 221s 11s/step - loss: 0.6512 - acc: 0.6297 - val_loss: 0.6152 - val_acc: 0.6857
Epoch 2/20
20/20 [==============================] - 32s 2s/step - loss: 0.5272 - acc: 0.7456 - val_loss: 0.4999 - val_acc: 0.7571
Epoch 3/20
20/20 [==============================] - 11s 548ms/step - loss: 0.4351 - acc: 0.8225 - val_loss: 0.5180 - val_acc: 0.7286
Epoch 4/20
20/20 [==============================] - 7s 355ms/step - loss: 0.3883 - acc: 0.8237 - val_loss: 0.4021 - val_acc: 0.7929
Epoch 5/20
20/20 [==============================] - 9s 455ms/step - loss: 0.3572 - acc: 0.8237 - val_loss: 0.3913 - val_acc: 0.8143
Epoch 6/20
20/20 [==============================] - 9s 436ms/step - loss: 0.3046 - acc: 0.8564 - val_loss: 0.5003 - val_acc: 0.7714
Epoch 7/20
20/20 [==============================] - 8s 403ms/step - loss: 0.3008 - acc: 0.8850 - val_loss: 0.3529 - val_acc: 0.8286
Epoch 8/20
20/20 [==============================] - 9s 453ms/step - loss: 0.2

In [26]:
from sklearn.metrics import f1_score

# Generate predictions for the validation set
validation_steps = len(validation_generator)
predictions = model3.predict_generator(validation_generator, steps=validation_steps)

# Convert predictions to binary values (0 or 1)
binary_predictions = np.round(predictions).reshape(-1)

# Get true labels for the validation set
true_labels = validation_generator.classes

# Calculate F1 score
f1score = f1_score(true_labels, binary_predictions)

print("F1 Score:", f1score)

<ipython-input-26-8e9c42b126ae>:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model3.predict_generator(validation_generator, steps=validation_steps)


F1 Score: 0.6666666666666666
